Demo

This is a demonstration of the tool in use with a langchain agent.
We will give the tool to a prebuilt langchain agent and ask it some questions.

In [1]:
import os
from tool import TapToolsAPIResults #import our taptools tool
import time # not explicitly used, but rate_limiter wasn't working w/o it

from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage
from langchain_core.rate_limiters import InMemoryRateLimiter

In [2]:
#setting rate limiter so we don't exceed anthropics rate limit on "basic" plan
# going to get a warning that rate_limiter is in beta annd subject to change
rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.02,  # requests per second
    check_every_n_seconds=60,  # Wake up every n seconds to check whether allowed to make a request,
    max_bucket_size=1,  # Controls the maximum burst size.
)

/var/folders/fh/7krdbwxx6pbbqjryxc9f6qjw0000gn/T/ipykernel_320/735258499.py:1: LangChainBetaWarning: Introduced in 0.2.24. API subject to change.
  rate_limiter = InMemoryRateLimiter(


In [3]:
if not os.environ.get("ANTHROPIC_API_KEY"):
  os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY")

taptools_key = os.getenv("TAPTOOLS_API_KEY")

In [4]:
model = ChatAnthropic(model="claude-3-5-sonnet-latest", temperature=0, rate_limiter=rate_limiter)

In [5]:
# instantiate tool
taptools_tool = TapToolsAPIResults()

#our prebuilt agent will need a list of tools
tools = [taptools_tool]

In [6]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [ ]:
response = agent_executor.invoke({"messages": [HumanMessage(content="What token has performed best in the last 30 days out of the top 50 tokens by market cap?")]})

print(response)